In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import plotly
import plotly.offline as py
import plotly.tools as tls   
import plotly.graph_objs as go
import colorlover as cl
from IPython.display import HTML
py.init_notebook_mode(connected=True)

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
mc = pd.read_csv('mc.csv', index_col='eid')
data = pd.read_csv('data.csv', index_col='eid')

In [4]:
# Global variables for plotting

# Change these if necessary:
run_default = 0 # = 0 for all runs
nbins_default = 100
normed_flag = False

# Shouldn't have to change these:
golden = (1 + 5 ** 0.5) / 2
fig_height = 680
fsize = 12

In [5]:
def create_stacked_plotly(var_name, data_range=None, nbins=nbins_default, run=run_default):
    # Get points and weights
    if run != 0:
        mc_var = mc.loc[mc['run']==run].loc[:,['eventlabel',var_name,'weight']]
        data_var = data.loc[data['run']==run].loc[:,var_name]
        if data_range: # Specific run with data cuts
            mc_weights = mc_var.loc[(mc_var[var_name] >= data_range[0]) & 
                                    (mc_var[var_name] <= data_range[1])].loc[:,['eventlabel','weight']]
            mc_var = mc_var.loc[(mc_var[var_name] >= data_range[0]) & 
                                (mc_var[var_name] <= data_range[1])].loc[:,['eventlabel',var_name]]
            data_var = [x for x in data_var if x >= data_range[0] and x <= data_range[1]]
        else: # Specific run without data cuts
            mc_weights = mc_var.loc[:,['eventlabel','weight']]
    else:
        mc_var = mc.loc[:,['eventlabel', var_name, 'weight']]
        data_var = data.loc[:,var_name]
        if data_range: # All runs with data cuts
            mc_weights = mc_var.loc[(mc_var[var_name] >= data_range[0]) & 
                                (mc_var[var_name] <= data_range[1])].loc[:,['eventlabel','weight']]
            mc_var = mc_var.loc[(mc_var[var_name] >= data_range[0]) & 
                                    (mc_var[var_name] <= data_range[1])].loc[:,['eventlabel',var_name]]
            data_var = [x for x in data_var if x >= data_range[0] and x <= data_range[1]]
        else: # All runs without data cuts
            mc_weights = mc_var.loc[:,['eventlabel','weight']]

    # Make histogram and record bin counts and edges
    data_counts, bin_edges = np.histogram(data_var, bins = nbins)
    bin_centers = (bin_edges[:-1] + bin_edges[1:])/2.
    dtau_counts,_=np.histogram(
        mc_var.loc[mc_var['eventlabel']==1].loc[:,var_name], 
        bins=bin_edges,
        weights=mc_weights.loc[mc_weights['eventlabel']==1].loc[:,'weight'],
    )
    dstartau_counts,_=np.histogram(
        mc_var.loc[mc_var['eventlabel']==2].loc[:,var_name], 
        bins=bin_edges,
        weights=mc_weights.loc[mc_weights['eventlabel']==2].loc[:,'weight'],
    )
    sl_counts,_=np.histogram(
        mc_var.loc[mc_var['eventlabel']==3].loc[:,var_name], 
        bins=bin_edges,
        weights=mc_weights.loc[mc_weights['eventlabel']==3].loc[:,'weight'],
    )
    had_counts,_=np.histogram(
        mc_var.loc[mc_var['eventlabel']==4].loc[:,var_name], 
        bins=bin_edges,
        weights=mc_weights.loc[mc_weights['eventlabel']==4].loc[:,'weight'],
    )
    cont_counts,_=np.histogram(
        mc_var.loc[mc_var['eventlabel']==5].loc[:,var_name], 
        bins=bin_edges,
        weights=mc_weights.loc[mc_weights['eventlabel']==5].loc[:,'weight']
    )
    
    # Set up plots
    colors = cl.scales['5']['qual']['Set2']
    mc1 = go.Bar(
        x = bin_centers,
        y = dtau_counts,
        name = 'DTau',
        #opacity = 0.6,
        marker = dict(
            color = colors[0],
            line = dict(
                color = 'black', 
                width = 1
            )
        )
    )
    mc2 = go.Bar(
        x = bin_centers,
        y = dstartau_counts,
        name = 'DStarTau',
        #opacity = 0.6,
        marker = dict(
            color = colors[1],
            line = dict(
                color = 'black', 
                width = 1
            )
        )
    )
    mc3 = go.Bar(
        x = bin_centers,
        y = sl_counts,
        name = 'SL',
        #opacity = 0.6,
        marker = dict(
            color = colors[2],
            line = dict(
                color = 'black', 
                width = 1
            )
        )
    )
    mc4 = go.Bar(
        x = bin_centers,
        y = had_counts,
        name = 'Had',
        #opacity = 0.6,
        marker = dict(
            color = colors[3],
            line = dict(
                color = 'black', 
                width = 1
            )
        )
    )
    mc5 = go.Bar(
        x = bin_centers,
        y = cont_counts,
        name = 'Cont',
        #opacity = 0.6,
        marker = dict(
            color = colors[4],
            line = dict(
                color = 'black', 
                width = 1
            )
        )
    )
    data1 = go.Scatter(
        x = bin_centers,
        y = data_counts,
        name = 'Data',
        mode = 'markers',
        marker = dict(
            color='black', 
            size=4
        ),
        error_y = dict(
            type='data', 
            array=[x**0.5 for x in data_counts], 
            visible=True, 
            color='black', 
            thickness=0.7
        )
    )
    
    # Calculate data-MC percent difference 
    mc_counts = [sum(n) for n in zip(dtau_counts, dstartau_counts, 
                                       sl_counts, had_counts, cont_counts)]
    diff_counts = [(x-y)*1./x for x,y in zip(data_counts, mc_counts)]
    diff_err = [np.sqrt(y*(x+y)/x**3) for x,y in zip(data_counts, mc_counts)]
    ylimlow = max([abs(x-y) for x,y in zip(diff_counts, diff_err)])
    ylimhigh = max([abs(x+y) for x,y in zip(diff_counts, diff_err)])
    ylim = ylimlow if ylimlow > ylimhigh else ylimhigh
    diff = go.Scatter(
        x = bin_centers,
        y = diff_counts,
        mode = 'markers',
        marker = dict(
            color='black', 
            size=4
        ),
        error_y = dict(
            type='data', 
            array=diff_err, 
            visible=True, 
            color='black', 
            thickness=0.7
        ),
        showlegend = False
    )
    
    # Set up the figure
    fig = tls.make_subplots(
        rows = 9,
        cols = 1,
        specs = [[{'rowspan':6}],
                 [None],
                 [None],
                 [None],
                 [None],
                 [None],
                 [None],
                 [{'rowspan':2}],
                 [None]],
        print_grid = False,
    )
    fig.append_trace(mc1, 1, 1)
    fig.append_trace(mc2, 1, 1)
    fig.append_trace(mc3, 1, 1)
    fig.append_trace(mc4, 1, 1)
    fig.append_trace(mc5, 1, 1)
    fig.append_trace(data1, 1, 1)
    fig.append_trace(diff, 8, 1)
    fig['layout'].update(
        height = fig_height,
        xaxis1 = dict(title = var_name),
        yaxis1 = dict(title = 'Counts'),
        xaxis2 = dict(title = var_name),
        yaxis2 = dict(
            title = 'Data-MC/Data', 
            range = [-1.*ylim, ylim]
        ),
        barmode = 'stack',
        bargap = 0
    )
    return py.iplot(fig)

In [44]:
create_stacked_plotly('signal_score', nbins=20, data_range=[0,0.1])

In [7]:
create_stacked_plotly('dstartau_score', data_range=[0.2,0.7])

In [8]:
create_stacked_plotly('mmiss2', data_range=[-3,30])

In [9]:
create_stacked_plotly('mmiss2prime', data_range=[-20,30])

In [10]:
create_stacked_plotly('eextra', data_range=[0,5])

In [11]:
create_stacked_plotly('costhetat')

In [12]:
create_stacked_plotly('tag_lp3', data_range=[0,2.3])

In [13]:
create_stacked_plotly('tag_cosby')

In [14]:
create_stacked_plotly('tag_costhetadl')

In [15]:
create_stacked_plotly('tag_isbdstar')

In [16]:
create_stacked_plotly('tag_dmass')

In [17]:
create_stacked_plotly('tag_deltam', data_range=[0,2])

In [18]:
create_stacked_plotly('tag_costhetadsoft', data_range=[-1.1, 1.1])

In [19]:
create_stacked_plotly('tag_softp3magcm', data_range=[-0.1, 1])

In [20]:
create_stacked_plotly('sig_hp3', data_range=[0,3])

In [21]:
create_stacked_plotly('sig_cosby', data_range=[-30,50])

In [22]:
create_stacked_plotly('sig_costhetadtau')

In [23]:
create_stacked_plotly('sig_vtxb')

In [24]:
create_stacked_plotly('sig_isbdstar')

In [25]:
create_stacked_plotly('sig_dmass', data_range=[1.5,1.95])

In [26]:
create_stacked_plotly('sig_deltam', data_range=[0, 1])

In [27]:
create_stacked_plotly('sig_costhetadsoft', data_range=[-1.1, 1])

In [28]:
create_stacked_plotly('sig_softp3magcm', data_range=[-0.5,1])

In [29]:
create_stacked_plotly('sig_hmass', data_range=[0,2])

In [30]:
create_stacked_plotly('sig_vtxh', data_range=[-0.5,2])

In [31]:
create_stacked_plotly('tag_dmode')

In [32]:
create_stacked_plotly('tag_dstarmode')

In [33]:
create_stacked_plotly('sig_dmode')

In [34]:
create_stacked_plotly('sig_dstarmode')

In [35]:
create_stacked_plotly('tag_l_epid')

In [36]:
create_stacked_plotly('tag_l_mupid')

In [37]:
create_stacked_plotly('sig_h_epid')

In [38]:
create_stacked_plotly('sig_h_mupid')

In [39]:
create_stacked_plotly('cand_score', data_range=[0,0.1], nbins=40)

In [40]:
create_stacked_plotly('ny', data_range=[0,50])

In [41]:
create_stacked_plotly('ntracks')

In [42]:
create_stacked_plotly('r2', data_range=[0,0.7])

In [43]:
create_stacked_plotly('r2all', data_range=[0,0.7])